## Train a model locally or remote

In [1]:
%cd ~/SageMaker/ezsm-ray-FM/dev/

/home/ec2-user/SageMaker/ezsm-ray-FM/dev


In [4]:
%pip uninstall -y ezsmdeploy
%pip install -e ../

Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Obtaining file:///home/ec2-user/SageMaker/easy-amazon-sagemaker-deployments
  Preparing metadata (setup.py) ... done
  Using cached sagemaker-2.173.0.tar.gz (854 kB)
  Preparing metadata (setup.py) ... done
  Using cached yaspin-0.16.0-py2.py3-none-any.whl (18 kB)
  Using cached shortuuid-1.0.1-py3-none-any.whl (7.5 kB)
  Using cached sagemaker_studio_image_build-0.5.0.tar.gz (11 kB)
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.1/670.1 kB 47.8 MB/s eta 0:00:00
  Created wheel for sagemaker: filename=sagemaker-2.173.0-py2.py3-none-any.whl size=1163282 sha256=6a8f12d456205b144f2e8075259e0e78e994435dfb7d35749f4cb3f4f2fe34e2
  Stored in directory: /home/ec2-user/.cache/pip/wheels/60/36/51/bd95494827eef1c60ac3a0a49b0c95051fe45d5ebe9a7b5e5f
  Created wheel for sagemaker-studio-image-build: filename=sa

### Note: you may need to restart the kernel to use updated packages.

## Testing foundation model 

In [3]:
# %pip uninstall -y sagemaker
# %pip install --upgrade sagemaker==2.148.0 --no-cache-dir

In [4]:
import sagemaker
sagemaker.__version__

'2.148.0'

In [5]:
!pip show ezsmdeploy

Name: ezsmdeploy
Version: 1.96.dev0
Summary: SageMaker custom deployments made easy
Home-page: https://pypi.python.org/pypi/ezsmdeploy
Author: Shreyas Subramanian
Author-email: subshrey@amazon.com
License: MIT
Location: /home/ec2-user/SageMaker/easy-amazon-sagemaker-deployments
Editable project location: /home/ec2-user/SageMaker/easy-amazon-sagemaker-deployments
Requires: boto3, sagemaker, sagemaker-studio-image-build, shortuuid, yaspin
Required-by: 


In [1]:
import ezsmdeploy

In [2]:
ezsmdeploy.utils().list_foundation_models()

List of foundation models in Jumpstart: 

huggingface-text2text-bart4csc-base-chinese
huggingface-text2text-bigscience-t0pp
huggingface-text2text-bigscience-t0pp-bnb-int8
huggingface-text2text-bigscience-t0pp-fp16
huggingface-text2text-flan-t5-base
huggingface-text2text-flan-t5-base-samsum
huggingface-text2text-flan-t5-large
huggingface-text2text-flan-t5-small
huggingface-text2text-flan-t5-xl
huggingface-text2text-flan-t5-xxl
huggingface-text2text-flan-t5-xxl-bnb-int8
huggingface-text2text-flan-t5-xxl-fp16
huggingface-text2text-flan-ul2-bf16
huggingface-text2text-pegasus-paraphrase
huggingface-text2text-qcpg-sentences
huggingface-text2text-t5-one-line-summary


In [ ]:
ezonsm = ezsmdeploy.Deploy(model = "huggingface-text2text-flan-t5-xxl-fp16",
                           foundation_model=True)

0:00:00.158649 | created model(s). Now deploying on ml.g5.12xlarge
●∙∙ ---------

### Set serializer if needed

In [ ]:
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

ezonsm.predictor.serializer = JSONSerializer()
ezonsm.predictor.deserializer = JSONDeserializer()

### Indvidual Utterance testing

In [17]:
payload = {
    "text_inputs": "<human>: Tell me the steps to make a pizza\n<bot>:",
    "max_length": 10,
    "max_time": 10,
    "top_k": 5,
    "top_p": 0.5,
    "do_sample": True}

response = ezonsm.predictor.predict(payload)

response

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (0) from primary with message "Your invocation timed out while waiting for a response from container primary. Review the latency metrics for each container in Amazon CloudWatch, resolve the issue, and try again.". See https://us-east-1.console.aws.amazon.com/cloudwatch/home?region=us-east-1#logEventViewer:group=/aws/sagemaker/Endpoints/ezsm-foundation-endpoint-i3h2m8cmsznfafqvo5tpjk in account 716845917484 for more information.

### Don't leave resources running

In [19]:
ezonsm.predictor.delete_endpoint()

ClientError: An error occurred (ValidationException) when calling the DeleteEndpointConfig operation: Could not find endpoint configuration "arn:aws:sagemaker:us-east-1:716845917484:endpoint-config/ezsm-foundation-endpoint-i3h2m8cmsznfafqvo5tpjk".